In [2]:
# BIBLIOTECAS NECESSÁRIAS AO MODELO DE CLASSIFICAÇÃO
from sklearn.datasets import fetch_openml

# Baixando o conjunto de dados

In [6]:
# Dataset mnist
mnist= fetch_openml('mnist_784',version=1,as_frame=False)
mnist.keys()
display('data','target', 'feature_names', 'DESCR', 'details', 
            'categories', 'url')

'data'

'target'

'feature_names'

'DESCR'

'details'

'categories'

'url'

In [8]:
# conferir os arrays
x, y = mnist['data'], mnist['target']
x.shape

(70000, 784)

In [9]:
y.shape

(70000,)